## **Cargar librerias y explorar los datos**

In [4]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import swifter
import dask.dataframe as dd

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\migue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import pandas as pd

# Cargar el dataset (ajusta la ruta si es necesario)
df = pd.read_csv("Reviews.csv")

# Mostrar las primeras filas
df.head()

In [ ]:
# Revisar informacion general del dataset
df.info()

In [ ]:
# vemos distribucion de las calificaciones
df["Score"].value_counts()

In [ ]:
# Ejemplo de reseñas
df[["Score", "Text"]].sample(5)

In [18]:
df = pd.read_csv('Reviews.csv')
columnas_a_eliminar = ['ProductId', 'UserId', 'ProfileName', 'Time', 'Summary']
df = df.drop(columnas_a_eliminar, axis=1)

print(df.columns) # Para verificar que las columnas se eliminaron

# guardar los datos procesados
df.to_csv('datos_preparados.csv', index=False) # index=False evita guardar el índice del DataFrame

Index(['Id', 'HelpfulnessNumerator', 'HelpfulnessDenominator', 'Score',
       'Text'],
      dtype='object')


## **Limpieza y Preprocesamiento del Texto**

In [ ]:
ddf = dd.read_csv('datos_preparados.csv')

def clean_text(text):
    text = ''.join([char.lower() for char in text if char.isalnum() or char.isspace()])
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

ddf['CleanedText'] = ddf['Text'].map(clean_text, meta=('CleanedText', 'object'))

ddf['Sentiment'] = ddf['Score'].apply(lambda score: 'positive' if score > 3 else 'negative', meta=('Sentiment', 'object'))

df = ddf.compute()


In [ ]:
# Opcion mas optima para guardar los datos procesados
df.to_parquet('datos_procesados.parquet', index=False)

## **Creacion del modelo**

In [ ]:
# Volvemos a cargar los datos ya procesados
df = pd.read_parquet('datos_procesados.parquet') 

In [ ]:
# División de los Datos para el modelo
X_train, X_test, y_train, y_test = train_test_split(df['CleanedText'], df['Sentiment'], test_size=0.2, random_state=42)

In [ ]:
# Convertimos el texto en una representación numérica usando TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

## **Entrenamiento del modelo**

In [10]:
# Entrenar modelo de regresión logística para la clasificación de sentimientos
model = LogisticRegression()
model.fit(X_train_vec, y_train)

LogisticRegression()

In [11]:
y_pred = model.predict(X_test_vec)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

print(classification_report(y_test, y_pred))

Accuracy: 0.9013202452260953
              precision    recall  f1-score   support

    negative       0.83      0.69      0.75     24666
    positive       0.92      0.96      0.94     89025

    accuracy                           0.90    113691
   macro avg       0.87      0.83      0.85    113691
weighted avg       0.90      0.90      0.90    113691



In [ ]:
# Guardamos el modelo entrenado

In [12]:
import joblib

# Suponiendo que 'model' es tu modelo entrenado
joblib.dump(model, 'modelo_sentimientos.joblib')
joblib.dump(vectorizer, 'vectorizador.joblib')

['vectorizador.joblib']

## **Prueba del modelo**

In [13]:
import joblib

modelo_cargado = joblib.load('modelo_sentimientos.joblib')
vectorizador_cargado = joblib.load('vectorizador.joblib')

In [14]:
nuevos_datos = ["This product is amazing!", "I'm very disappointed."]
nuevos_datos_vectorizados = vectorizador_cargado.transform(nuevos_datos)
predicciones = modelo_cargado.predict(nuevos_datos_vectorizados)
print(predicciones)

['positive' 'negative']
